In [1]:
import pandas as pd
from pandas import ExcelWriter
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from sklearn.linear_model import Lasso
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

from IPython.display import display # Allows the use of display() for DataFrames

import matplotlib.pyplot as plt
from sklearn.svm import SVC
import seaborn as sns
sns.set(style="ticks")

import pickle

# Pretty display for notebooks
%matplotlib inline

from datetime import date


In [2]:
# load the models from disk

filename1 = 'sales_fcst_rf.sav'
rf = pickle.load(open(filename1, 'rb'))

filename2 = 'sales_fcst_rf_intl_together.sav'
rf_int_t = pickle.load(open(filename2, 'rb'))

filename3 = 'sales_fcst_gb.sav'
gb = pickle.load(open(filename3, 'rb'))

filename4 = 'sales_fcst_gb_intl_together.sav'
gb_int_t = pickle.load(open(filename4, 'rb'))


In [3]:
redshift_endpoint = 'bi-dw-instance.cqutp4iwqhnr.us-east-1.redshift.amazonaws.com'
redshift_user = 'ds_derek'
redshift_pass = 'Z00mdwh36762'
port = 5439
dbname = 'zoomdwhdb'

In [4]:
from sqlalchemy import create_engine
from sqlalchemy import text
engine_string = "postgresql+psycopg2://%s:%s@%s:%d/%s" \
% (redshift_user, redshift_pass, redshift_endpoint, port, dbname)
engine = create_engine(engine_string)

In [5]:
sql = '''
-----------
-----------
-----------
----------- TESTING
-----------
-----------
-----------


WITH a AS 
--- Opportunities
(SELECT a.dt,
       c.fy_quarter,
       a.dt - c.qtr_start + 1 AS day_of_qtr,
       case when d.sales_div_clean is null then 'UnDefined' else d.sales_div_clean end AS sales_div,
       SUM(CASE WHEN a.stagename = '1. Qualification' THEN 1 ELSE 0 END) AS stage_1_count,
       SUM(CASE WHEN a.stagename = '1. Qualification' THEN a.amount_usd ELSE 0 END) AS stage_1_amount,
       SUM(CASE WHEN a.stagename = '2. Discovery' THEN 1 ELSE 0 END) AS stage_2_count,
       SUM(CASE WHEN a.stagename = '2. Discovery' THEN a.amount_usd ELSE 0 END) AS stage_2_amount,
       SUM(CASE WHEN a.stagename = '3. Solution' THEN 1 ELSE 0 END) AS stage_3_count,
       SUM(CASE WHEN a.stagename = '3. Solution' THEN a.amount_usd ELSE 0 END) AS stage_3_amount,
       SUM(CASE WHEN a.stagename = '4. POC' THEN 1 ELSE 0 END) AS stage_4_count,
       SUM(CASE WHEN a.stagename = '4. POC' THEN a.amount_usd ELSE 0 END) AS stage_4_amount,
       SUM(CASE WHEN a.stagename = '5. Contract' THEN 1 ELSE 0 END) AS stage_5_count,
       SUM(CASE WHEN a.stagename = '5. Contract' THEN a.amount_usd ELSE 0 END) AS stage_5_amount,
       SUM(CASE WHEN a.stagename IN ('3. Solution','4. POC','5. Contract') and a.amount_usd > 10000 THEN a.amount_usd ELSE 0 END) stage345_large_deals,
       SUM(CASE WHEN a.stagename IN ('4. POC','5. Contract') and a.amount_usd > 10000 THEN a.amount_usd ELSE 0 END) stage45_large_deals,
       SUM(CASE WHEN a.stagename IN ('5. Contract') and a.amount_usd > 10000 THEN a.amount_usd ELSE 0 END) stage5_large_deals
FROM  (SELECT a.*, b.rate, a.amount / b.rate AS amount_usd ------ opps in USD
        FROM src_sfdc.opportunity_history a
        LEFT JOIN src_zuora.currency b ON (case when a.currencyisocode is null then 'USD' else a.currencyisocode end) = b.alphabeticcode
        where a.dt - a.lastactivitydate::date < 120 --- only opps that were touched within 120 days
        ) a 
  LEFT JOIN src_sfdc.account b ON a.accountid = b.id
  LEFT JOIN src_config.zoom_quarter_mapping c
         ON a.dt BETWEEN c.qtr_start
        AND c.qtr_end
  LEFT JOIN lab.dw_20190311_sales_owner_division_clean_up d ON a.owner_division__c = d.sales_div_dirty
WHERE 1 = 1
AND   a.isdeleted = FALSE
AND   a.isclosed = FALSE
AND   a.amount_usd > 0
AND   a.closedate between c.qtr_start AND c.qtr_end ---- only opps closing from the given quarter
AND   a.dt >= (SELECT qtr_start
             FROM src_config.zoom_quarter_mapping
             WHERE CURRENT_DATE BETWEEN qtr_start AND qtr_end)
GROUP BY 1,
         2,
         3,
         4),
         
b AS
---- QTD bookings
(SELECT booking_date__c as dt,
       fy_quarter,
       case when sales_div_clean is null then 'UnDefined' else sales_div_clean end as sales_div,
       SUM(bookings) OVER (PARTITION BY fy_quarter, sales_div_clean ORDER BY booking_date__c ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS bookings_qtd
FROM (SELECT a.booking_date__c,
             b.fy_quarter,
             c.sales_div_clean,
             SUM(Amount__c) AS bookings
      FROM src_sfdc.bookings a
        LEFT JOIN src_config.zoom_quarter_mapping b
               ON a.booking_date__c BETWEEN b.qtr_start
              AND b.qtr_end
        LEFT JOIN lab.dw_20190311_sales_owner_division_clean_up c ON a.owner_division__c = c.sales_div_dirty
      WHERE 1 = 1
      AND   isdeleted = FALSE
      AND   ((Order_Type__c IN ('New','New Order') AND Amount__c >= 17) 
            OR (Order_Type__c = 'Upsell' AND Amount__c >= 0) 
            OR (Order_Type__c IN ('New','New Order') AND Amount__c < 17 AND Coupon__c <> '' AND coupon__c IS NOT NULL) 
            OR (bookingexception__c = 'Y'))
      AND   LOWER(owner_name) NOT LIKE '%integration%'
      AND   account__c <> ''
      AND   account__c IS NOT NULL
      AND   a.booking_date__c >= '2018-02-01'
      AND   a.booking_date__c >= (SELECT qtr_start
                                FROM src_config.zoom_quarter_mapping
                                WHERE CURRENT_DATE BETWEEN qtr_start AND qtr_end)
      GROUP BY 1,
               2,
               3)
ORDER BY 3,1),

c AS
---- Bookings total for the quarter
(SELECT b.fy_quarter,
       case when c.sales_div_clean is null then 'UnDefined' else c.sales_div_clean end as sales_div,
       SUM(Amount__c) AS direct_bookings
FROM src_sfdc.bookings a
  LEFT JOIN src_config.zoom_quarter_mapping b
         ON a.booking_date__c BETWEEN b.qtr_start
        AND b.qtr_end
  LEFT JOIN lab.dw_20190311_sales_owner_division_clean_up c ON a.owner_division__c = c.sales_div_dirty
WHERE 1 = 1
AND   isdeleted = FALSE
AND   ((Order_Type__c IN ('New','New Order') AND Amount__c >= 17) 
      OR (Order_Type__c = 'Upsell' AND Amount__c >= 0) 
      OR (Order_Type__c IN ('New','New Order') AND Amount__c < 17 AND Coupon__c <> '' AND coupon__c IS NOT NULL) 
      OR (bookingexception__c = 'Y'))
AND   lower(owner_name) NOT LIKE '%integration%'
AND   account__c <> ''
AND   account__c IS NOT NULL
AND   a.booking_date__c >= '2018-02-01'
AND   a.booking_date__c >= (SELECT qtr_start
                          FROM src_config.zoom_quarter_mapping
                          WHERE CURRENT_DATE BETWEEN qtr_start AND qtr_end)
GROUP BY 1,2
ORDER BY 1,2),

d AS
(SELECT b.fy_quarter,
       case when c.sales_div_clean is null then 'UnDefined' else c.sales_div_clean end as sales_div,
       SUM(quota__c) AS quota
FROM src_sfdc.quota a
  LEFT JOIN src_config.zoom_quarter_mapping b ON a.start_date__c::date = b.qtr_start
  LEFT JOIN lab.dw_20190311_sales_owner_division_clean_up c ON a.email__c = c.sales_div_dirty
WHERE quota_owner_type__c = 'Segment'
and quota__c > 0
GROUP BY 1,2)

SELECT a.dt,
       a.fy_quarter,
       a.day_of_qtr,
       a.sales_div,
       d.quota,
       a.stage_1_count,
       a.stage_1_amount,
       a.stage_2_count,
       a.stage_2_amount,
       a.stage_3_count,
       a.stage_3_amount,
       a.stage_4_count,
       a.stage_4_amount,
       a.stage_5_count,
       a.stage_5_amount,
       CASE WHEN b.bookings_qtd IS NULL AND MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) IS NULL THEN 0
            WHEN b.bookings_qtd IS NULL THEN MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) 
            ELSE b.bookings_qtd END as bookings_qtd,
       c.direct_bookings,
       case when a.fy_quarter like '%Q1' then a.day_of_qtr::float / 89 else a.day_of_qtr::float / 92 end as qtr_pct,
       a.stage_1_amount / d.quota AS s1_quota_ratio,
       a.stage_2_amount / d.quota AS s2_quota_ratio,
       a.stage_3_amount / d.quota AS s3_quota_ratio,
       a.stage_4_amount / d.quota AS s4_quota_ratio,
       a.stage_5_amount / d.quota AS s5_quota_ratio,
       (a.stage_1_amount + a.stage_2_amount + a.stage_3_amount + a.stage_4_amount + a.stage_5_amount +      
            CASE WHEN b.bookings_qtd IS NULL AND MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) IS NULL THEN 0
                 WHEN b.bookings_qtd IS NULL THEN MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) 
                 ELSE b.bookings_qtd END )/ d.quota AS s12345_bk_qtd_quota_ratio,
       (a.stage_2_amount + a.stage_3_amount + a.stage_4_amount + a.stage_5_amount +      
            CASE WHEN b.bookings_qtd IS NULL AND MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) IS NULL THEN 0
                 WHEN b.bookings_qtd IS NULL THEN MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) 
                 ELSE b.bookings_qtd END )/ d.quota AS s2345_bk_qtd_quota_ratio,
       (a.stage_3_amount + a.stage_4_amount + a.stage_5_amount +      
            CASE WHEN b.bookings_qtd IS NULL AND MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) IS NULL THEN 0
                 WHEN b.bookings_qtd IS NULL THEN MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) 
                 ELSE b.bookings_qtd END )/ d.quota AS s345_bk_qtd_quota_ratio,
       (a.stage_4_amount + a.stage_5_amount +      
            CASE WHEN b.bookings_qtd IS NULL AND MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) IS NULL THEN 0
                 WHEN b.bookings_qtd IS NULL THEN MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) 
                 ELSE b.bookings_qtd END )/ d.quota AS s45_bk_qtd_quota_ratio,
       (a.stage_5_amount +      
            CASE WHEN b.bookings_qtd IS NULL AND MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) IS NULL THEN 0
                 WHEN b.bookings_qtd IS NULL THEN MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) 
                 ELSE b.bookings_qtd END )/ d.quota AS s5_bk_qtd_quota_ratio,

       (a.stage345_large_deals / d.quota) AS s345_lg_deal_quota_ratio,
       (a.stage45_large_deals / d.quota) AS s45_lg_deal_quota_ratio,
       (a.stage5_large_deals / d.quota) AS s5_lg_deal_quota_ratio,

       (CASE WHEN b.bookings_qtd IS NULL AND MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) IS NULL THEN 0
                 WHEN b.bookings_qtd IS NULL THEN MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) 
                 ELSE b.bookings_qtd END ) / d.quota as bookings_pct_qtd,
       c.direct_bookings / d.quota as bookings_pct_finish
FROM a
  LEFT JOIN b
         ON a.dt = b.dt
        AND a.sales_div = b.sales_div
        AND a.fy_quarter = b.fy_quarter
  LEFT JOIN c
         ON a.sales_div = c.sales_div
        AND a.fy_quarter = c.fy_quarter
  LEFT JOIN d
         ON a.sales_div = d.sales_div
        AND a.fy_quarter = d.fy_quarter
WHERE a.dt >= (SELECT qtr_start
                          FROM src_config.zoom_quarter_mapping
                          WHERE CURRENT_DATE BETWEEN qtr_start AND qtr_end)
AND a.sales_div NOT IN ('Channel', 'ISV', 'Online Team', 'UnDefined', 'API')
ORDER BY 4,1,2,3;'''
raw_data = pd.read_sql_query(text(sql), engine)
raw_data.to_csv("/Users/derekwang/Desktop/Python/Sales Forecast/data_testing_curr_quarter.csv")

In [6]:
sql = '''
-----------
-----------
-----------
----------- TESTING
-----------
-----------
-----------


WITH a AS 
--- Opportunities
(SELECT a.dt,
       c.fy_quarter,
       a.dt - c.qtr_start + 1 AS day_of_qtr,
       case when d.sales_div_clean is null then 'UnDefined' else d.sales_div_clean end AS sales_div,
       SUM(CASE WHEN a.stagename = '1. Qualification' THEN 1 ELSE 0 END) AS stage_1_count,
       SUM(CASE WHEN a.stagename = '1. Qualification' THEN a.amount_usd ELSE 0 END) AS stage_1_amount,
       SUM(CASE WHEN a.stagename = '2. Discovery' THEN 1 ELSE 0 END) AS stage_2_count,
       SUM(CASE WHEN a.stagename = '2. Discovery' THEN a.amount_usd ELSE 0 END) AS stage_2_amount,
       SUM(CASE WHEN a.stagename = '3. Solution' THEN 1 ELSE 0 END) AS stage_3_count,
       SUM(CASE WHEN a.stagename = '3. Solution' THEN a.amount_usd ELSE 0 END) AS stage_3_amount,
       SUM(CASE WHEN a.stagename = '4. POC' THEN 1 ELSE 0 END) AS stage_4_count,
       SUM(CASE WHEN a.stagename = '4. POC' THEN a.amount_usd ELSE 0 END) AS stage_4_amount,
       SUM(CASE WHEN a.stagename = '5. Contract' THEN 1 ELSE 0 END) AS stage_5_count,
       SUM(CASE WHEN a.stagename = '5. Contract' THEN a.amount_usd ELSE 0 END) AS stage_5_amount,
       SUM(CASE WHEN a.stagename IN ('3. Solution','4. POC','5. Contract') and a.amount_usd > 10000 THEN a.amount_usd ELSE 0 END) stage345_large_deals,
       SUM(CASE WHEN a.stagename IN ('4. POC','5. Contract') and a.amount_usd > 10000 THEN a.amount_usd ELSE 0 END) stage45_large_deals,
       SUM(CASE WHEN a.stagename IN ('5. Contract') and a.amount_usd > 10000 THEN a.amount_usd ELSE 0 END) stage5_large_deals
FROM  (SELECT a.*, b.rate, a.amount / b.rate AS amount_usd ------ opps in USD
        FROM src_sfdc.opportunity_history a
        LEFT JOIN src_zuora.currency b ON (case when a.currencyisocode is null then 'USD' else a.currencyisocode end) = b.alphabeticcode
        where a.dt - a.lastactivitydate::date < 120 --- only opps that were touched within 120 days
        ) a 
  LEFT JOIN src_sfdc.account b ON a.accountid = b.id
  LEFT JOIN src_config.zoom_quarter_mapping c
         ON a.dt BETWEEN c.qtr_start
        AND c.qtr_end
  LEFT JOIN lab.dw_20190311_sales_owner_division_clean_up_intl_as_one d ON a.owner_division__c = d.sales_div_dirty
WHERE 1 = 1
AND   a.isdeleted = FALSE
AND   a.isclosed = FALSE
AND   a.amount_usd > 0
AND   a.closedate between c.qtr_start AND c.qtr_end ---- only opps closing from the given quarter
AND   a.dt >= (SELECT qtr_start
             FROM src_config.zoom_quarter_mapping
             WHERE CURRENT_DATE BETWEEN qtr_start AND qtr_end)
GROUP BY 1,
         2,
         3,
         4),
         
b AS
---- QTD bookings
(SELECT booking_date__c as dt,
       fy_quarter,
       case when sales_div_clean is null then 'UnDefined' else sales_div_clean end as sales_div,
       SUM(bookings) OVER (PARTITION BY fy_quarter, sales_div_clean ORDER BY booking_date__c ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS bookings_qtd
FROM (SELECT a.booking_date__c,
             b.fy_quarter,
             c.sales_div_clean,
             SUM(Amount__c) AS bookings
      FROM src_sfdc.bookings a
        LEFT JOIN src_config.zoom_quarter_mapping b
               ON a.booking_date__c BETWEEN b.qtr_start
              AND b.qtr_end
        LEFT JOIN lab.dw_20190311_sales_owner_division_clean_up_intl_as_one c ON a.owner_division__c = c.sales_div_dirty
      WHERE 1 = 1
      AND   isdeleted = FALSE
      AND   ((Order_Type__c IN ('New','New Order') AND Amount__c >= 17) 
            OR (Order_Type__c = 'Upsell' AND Amount__c >= 0) 
            OR (Order_Type__c IN ('New','New Order') AND Amount__c < 17 AND Coupon__c <> '' AND coupon__c IS NOT NULL) 
            OR (bookingexception__c = 'Y'))
      AND   LOWER(owner_name) NOT LIKE '%integration%'
      AND   account__c <> ''
      AND   account__c IS NOT NULL
      AND   a.booking_date__c >= '2018-02-01'
      AND   a.booking_date__c >= (SELECT qtr_start
                                FROM src_config.zoom_quarter_mapping
                                WHERE CURRENT_DATE BETWEEN qtr_start AND qtr_end)
      GROUP BY 1,
               2,
               3)
ORDER BY 3,1),

c AS
---- Bookings total for the quarter
(SELECT b.fy_quarter,
       case when c.sales_div_clean is null then 'UnDefined' else c.sales_div_clean end as sales_div,
       SUM(Amount__c) AS direct_bookings
FROM src_sfdc.bookings a
  LEFT JOIN src_config.zoom_quarter_mapping b
         ON a.booking_date__c BETWEEN b.qtr_start
        AND b.qtr_end
  LEFT JOIN lab.dw_20190311_sales_owner_division_clean_up_intl_as_one c ON a.owner_division__c = c.sales_div_dirty
WHERE 1 = 1
AND   isdeleted = FALSE
AND   ((Order_Type__c IN ('New','New Order') AND Amount__c >= 17) 
      OR (Order_Type__c = 'Upsell' AND Amount__c >= 0) 
      OR (Order_Type__c IN ('New','New Order') AND Amount__c < 17 AND Coupon__c <> '' AND coupon__c IS NOT NULL) 
      OR (bookingexception__c = 'Y'))
AND   lower(owner_name) NOT LIKE '%integration%'
AND   account__c <> ''
AND   account__c IS NOT NULL
AND   a.booking_date__c >= '2018-02-01'
AND   a.booking_date__c >= (SELECT qtr_start
                          FROM src_config.zoom_quarter_mapping
                          WHERE CURRENT_DATE BETWEEN qtr_start AND qtr_end)
GROUP BY 1,2
ORDER BY 1,2),

d AS
(SELECT b.fy_quarter,
       case when c.sales_div_clean is null then 'UnDefined' else c.sales_div_clean end as sales_div,
       SUM(quota__c) AS quota
FROM src_sfdc.quota a
  LEFT JOIN src_config.zoom_quarter_mapping b ON a.start_date__c::date = b.qtr_start
  LEFT JOIN lab.dw_20190311_sales_owner_division_clean_up_intl_as_one c ON a.email__c = c.sales_div_dirty
WHERE quota_owner_type__c = 'Segment'
and quota__c > 0
GROUP BY 1,2)

SELECT a.dt,
       a.fy_quarter,
       a.day_of_qtr,
       a.sales_div,
       d.quota,
       a.stage_1_count,
       a.stage_1_amount,
       a.stage_2_count,
       a.stage_2_amount,
       a.stage_3_count,
       a.stage_3_amount,
       a.stage_4_count,
       a.stage_4_amount,
       a.stage_5_count,
       a.stage_5_amount,
       CASE WHEN b.bookings_qtd IS NULL AND MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) IS NULL THEN 0
            WHEN b.bookings_qtd IS NULL THEN MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) 
            ELSE b.bookings_qtd END as bookings_qtd,
       c.direct_bookings,
       case when a.fy_quarter like '%Q1' then a.day_of_qtr::float / 89 else a.day_of_qtr::float / 92 end as qtr_pct,
       a.stage_1_amount / d.quota AS s1_quota_ratio,
       a.stage_2_amount / d.quota AS s2_quota_ratio,
       a.stage_3_amount / d.quota AS s3_quota_ratio,
       a.stage_4_amount / d.quota AS s4_quota_ratio,
       a.stage_5_amount / d.quota AS s5_quota_ratio,
       (a.stage_1_amount + a.stage_2_amount + a.stage_3_amount + a.stage_4_amount + a.stage_5_amount +      
            CASE WHEN b.bookings_qtd IS NULL AND MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) IS NULL THEN 0
                 WHEN b.bookings_qtd IS NULL THEN MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) 
                 ELSE b.bookings_qtd END )/ d.quota AS s12345_bk_qtd_quota_ratio,
       (a.stage_2_amount + a.stage_3_amount + a.stage_4_amount + a.stage_5_amount +      
            CASE WHEN b.bookings_qtd IS NULL AND MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) IS NULL THEN 0
                 WHEN b.bookings_qtd IS NULL THEN MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) 
                 ELSE b.bookings_qtd END )/ d.quota AS s2345_bk_qtd_quota_ratio,
       (a.stage_3_amount + a.stage_4_amount + a.stage_5_amount +      
            CASE WHEN b.bookings_qtd IS NULL AND MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) IS NULL THEN 0
                 WHEN b.bookings_qtd IS NULL THEN MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) 
                 ELSE b.bookings_qtd END )/ d.quota AS s345_bk_qtd_quota_ratio,
       (a.stage_4_amount + a.stage_5_amount +      
            CASE WHEN b.bookings_qtd IS NULL AND MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) IS NULL THEN 0
                 WHEN b.bookings_qtd IS NULL THEN MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) 
                 ELSE b.bookings_qtd END )/ d.quota AS s45_bk_qtd_quota_ratio,
       (a.stage_5_amount +      
            CASE WHEN b.bookings_qtd IS NULL AND MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) IS NULL THEN 0
                 WHEN b.bookings_qtd IS NULL THEN MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) 
                 ELSE b.bookings_qtd END )/ d.quota AS s5_bk_qtd_quota_ratio,

       (a.stage345_large_deals / d.quota) AS s345_lg_deal_quota_ratio,
       (a.stage45_large_deals / d.quota) AS s45_lg_deal_quota_ratio,
       (a.stage5_large_deals / d.quota) AS s5_lg_deal_quota_ratio,

       (CASE WHEN b.bookings_qtd IS NULL AND MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) IS NULL THEN 0
                 WHEN b.bookings_qtd IS NULL THEN MAX(b.bookings_qtd) OVER (PARTITION BY a.fy_quarter, a.sales_div ORDER BY a.sales_div, a.dt ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) 
                 ELSE b.bookings_qtd END ) / d.quota as bookings_pct_qtd,
       c.direct_bookings / d.quota as bookings_pct_finish
FROM a
  LEFT JOIN b
         ON a.dt = b.dt
        AND a.sales_div = b.sales_div
        AND a.fy_quarter = b.fy_quarter
  LEFT JOIN c
         ON a.sales_div = c.sales_div
        AND a.fy_quarter = c.fy_quarter
  LEFT JOIN d
         ON a.sales_div = d.sales_div
        AND a.fy_quarter = d.fy_quarter
WHERE a.dt >= (SELECT qtr_start
                          FROM src_config.zoom_quarter_mapping
                          WHERE CURRENT_DATE BETWEEN qtr_start AND qtr_end)
AND a.sales_div NOT IN ('Channel', 'ISV', 'Online Team', 'UnDefined', 'API')
ORDER BY 4,1,2,3;'''
raw_data = pd.read_sql_query(text(sql), engine)
raw_data.to_csv("/Users/derekwang/Desktop/Python/Sales Forecast/data_testing_curr_quarter_intl_together.csv")

In [7]:
#### predict for current quarter ####

# Load the current_quarter_data 
curr_data = pd.read_csv("/Users/derekwang/Desktop/Python/Sales Forecast/data_testing_curr_quarter.csv")
curr_data = curr_data.drop([curr_data.columns[0]] , axis='columns')

# Define outcome
outcomes_raw_c = curr_data[['bookings_pct_finish']]

# Define features = removing outcome, dt, and fy_quarter label
features_raw_c1 = curr_data.drop(['direct_bookings', 'bookings_pct_finish'], axis = 1)

features_raw_c = features_raw_c1.drop(['dt', 
                                'fy_quarter', 
                                'day_of_qtr', 
                                'quota', 
                                'stage_1_count', 
                                'stage_1_amount', 
                                'stage_2_count', 
                                'stage_2_amount', 
                                'stage_3_count', 
                                'stage_3_amount', 
                                'stage_4_count', 
                                'stage_4_amount', 
                                'stage_5_count', 
                                'stage_5_amount', 
                                'bookings_qtd',
                                's1_quota_ratio', 's2_quota_ratio', 's3_quota_ratio', 's4_quota_ratio','s5_quota_ratio'
#                                       ,'sales_div'   
                                      ], axis = 1)

# fill in NA's with 0 - FEATURES
features_c = features_raw_c.fillna(0.0)

# fill in NA's with 0 - OUTCOMES
outcomes_c = outcomes_raw_c.fillna(0.0)

# One-hot encoding
X_curr = pd.get_dummies(features_c)

# get rid of blanks in headers
X_curr.columns = X_curr.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

print(X_curr.columns)

models = [rf, gb]
for x in models:
    print("")
    print(x)
    y_curr = x.predict(X_curr)
    y_curr = y_curr.flatten()
    y_curr = pd.DataFrame(y_curr)
    # export prediciton
    dfc = pd.concat([features_raw_c1.reset_index(), y_curr.reset_index()], axis=1)
    if x == rf:
#         dfc.to_csv("/Users/derekwang/Desktop/Python/Sales Forecast/curr_quarter_prediction__2_random_forest_" + str(date.today()) + ".csv", sep=',')
        writer = ExcelWriter("/Users/derekwang/Desktop/Python/Sales Forecast/curr_quarter_prediction__2_random_forest_" + str(date.today()) + ".xlsx")
        dfc.to_excel(writer,'Sheet1')
        writer.save()
        print("")
        print("DONE - rf")
    if x == gb:
#         dfc.to_csv("/Users/derekwang/Desktop/Python/Sales Forecast/curr_quarter_prediction__3_gradient_boost_" + str(date.today()) + ".csv", sep=',')
        writer = ExcelWriter("/Users/derekwang/Desktop/Python/Sales Forecast/curr_quarter_prediction__3_gradient_boost_" + str(date.today()) + ".xlsx")
        dfc.to_excel(writer,'Sheet1')
        writer.save()
        print("")
        print("DONE - gb")
        

Index(['qtr_pct', 's12345_bk_qtd_quota_ratio', 's2345_bk_qtd_quota_ratio',
       's345_bk_qtd_quota_ratio', 's45_bk_qtd_quota_ratio',
       's5_bk_qtd_quota_ratio', 's345_lg_deal_quota_ratio',
       's45_lg_deal_quota_ratio', 's5_lg_deal_quota_ratio', 'bookings_pct_qtd',
       'sales_div_comm', 'sales_div_comm-vast', 'sales_div_ed',
       'sales_div_ent', 'sales_div_gov', 'sales_div_healthcare',
       'sales_div_intl', 'sales_div_intl-anz', 'sales_div_intl-apac',
       'sales_div_intl-emea', 'sales_div_intl-uk', 'sales_div_majors',
       'sales_div_network_alliance', 'sales_div_smb', 'sales_div_smb-vast'],
      dtype='object')

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=3, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=Non

In [8]:
#### predict for current quarter ####

# Load the current_quarter_data 
curr_data = pd.read_csv("/Users/derekwang/Desktop/Python/Sales Forecast/data_testing_curr_quarter_intl_together.csv")
curr_data = curr_data.drop([curr_data.columns[0]] , axis='columns')

# Define outcome
outcomes_raw_c = curr_data[['bookings_pct_finish']]

# Define features = removing outcome, dt, and fy_quarter label
features_raw_c1 = curr_data.drop(['direct_bookings', 'bookings_pct_finish'], axis = 1)

features_raw_c = features_raw_c1.drop(['dt', 
                                'fy_quarter', 
                                'day_of_qtr', 
                                'quota', 
                                'stage_1_count', 
                                'stage_1_amount', 
                                'stage_2_count', 
                                'stage_2_amount', 
                                'stage_3_count', 
                                'stage_3_amount', 
                                'stage_4_count', 
                                'stage_4_amount', 
                                'stage_5_count', 
                                'stage_5_amount', 
                                'bookings_qtd',
                                's1_quota_ratio', 's2_quota_ratio', 's3_quota_ratio', 's4_quota_ratio','s5_quota_ratio'
#                                       ,'sales_div'   
                                      ], axis = 1)

# fill in NA's with 0 - FEATURES
features_c = features_raw_c.fillna(0.0)

# fill in NA's with 0 - OUTCOMES
# outcomes_c = outcomes_raw_c.fillna(0.0)

# One-hot encoding
X_curr = pd.get_dummies(features_c)

# get rid of blanks in headers
X_curr.columns = X_curr.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

print(X_curr.columns)



models = [rf_int_t, gb_int_t]
for x in models:
    print("")
    print(x)
    y_curr = x.predict(X_curr)
    y_curr = y_curr.flatten()
    y_curr = pd.DataFrame(y_curr)
    # export prediciton
    dfc = pd.concat([features_raw_c1.reset_index(), y_curr.reset_index()], axis=1)
    if x == rf_int_t:
#         dfc.to_csv("/Users/derekwang/Desktop/Python/Sales Forecast/curr_quarter_prediction__4_random_forest_intl_together_" + str(date.today()) + ".csv", sep=',')
        writer = ExcelWriter("/Users/derekwang/Desktop/Python/Sales Forecast/curr_quarter_prediction__4_random_forest_intl_together_" + str(date.today()) + ".xlsx")
        dfc.to_excel(writer,'Sheet1')
        writer.save()
        print("")
        print("DONE - rf_int_t")
    if x == gb_int_t:
#         dfc.to_csv("/Users/derekwang/Desktop/Python/Sales Forecast/curr_quarter_prediction__5_gradient_boost_intl_together_" + str(date.today()) + ".csv", sep=',')
        writer = ExcelWriter("/Users/derekwang/Desktop/Python/Sales Forecast/curr_quarter_prediction__5_gradient_boost_intl_together_" + str(date.today()) + ".xlsx")
        dfc.to_excel(writer,'Sheet1')
        writer.save()
        print("")
        print("DONE - gb_int_t")

Index(['qtr_pct', 's12345_bk_qtd_quota_ratio', 's2345_bk_qtd_quota_ratio',
       's345_bk_qtd_quota_ratio', 's45_bk_qtd_quota_ratio',
       's5_bk_qtd_quota_ratio', 's345_lg_deal_quota_ratio',
       's45_lg_deal_quota_ratio', 's5_lg_deal_quota_ratio', 'bookings_pct_qtd',
       'sales_div_comm', 'sales_div_comm-vast', 'sales_div_ed',
       'sales_div_ent', 'sales_div_gov', 'sales_div_healthcare',
       'sales_div_intl', 'sales_div_majors', 'sales_div_network_alliance',
       'sales_div_smb', 'sales_div_smb-vast'],
      dtype='object')

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=3, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

DONE - rf_int_t

GradientBoostingRegressor(alpha=0.9, criterion